In [ ]:
!pip install --upgrade setuptools
!pip install --upgrade gradio

In [ ]:
import json
import setuptools

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from dotenv import dotenv_values
from typechat import Failure, TypeChatValidator, create_language_model
from examples.healthData import schema as health
from examples.healthData.translator import TranslatorWithHistory

In [ ]:
health_instructions = """
Help me enter my health data step by step.
Ask specific questions to gather required and optional fields I have not already providedStop asking if I don't know the answer
Automatically fix my spelling mistakes
My health data may be complex: always record and return ALL of it.
Always return a response:
- If you don't understand what I say, ask a question.
- At least respond with an OK message.

"""

vals = dotenv_values()
model = create_language_model(vals)
validator = TypeChatValidator(health.HealthDataResponse)
translator = TranslatorWithHistory(model, validator, health.HealthDataResponse, additional_agent_instructions=health_instructions)

In [ ]:
import pandas

async def get_translation(message, history):
    result = await translator.translate(message)
    if isinstance(result, Failure):
        return f"Translation Failed ❌ \n Context: {result.message}"
    else:
        result = result.value
        output = f"Translation Succeeded! ✅\n"
        
        data = result.get("data", None)
        if data:
            df = pandas.DataFrame.from_dict(data)
            output += f"HealthData \n ``` {df.fillna('').to_markdown(tablefmt='grid')} \n ```  \n"

        message = result.get("message", None)
        not_translated = result.get("notTranslated", None)

        if message:
            output += f"\n📝: {message}"
            
        if not_translated:
            output += f"\n🤔: I did not understand\n {not_translated}" 
            
        return output


def get_examples():
    example_prompts = []
    with open('../examples/healthData/input.txt') as prompts_file:
        for line in prompts_file:
            example_prompts.append(line)
    return example_prompts


In [ ]:
import gradio as gr

gr.ChatInterface(get_translation, title="💉💊🤧 Health Data", examples=get_examples()).launch(debug=False)
